In [2]:
import pandas as pd
import pickle
import seaborn as sns
import numpy as np
path = '##'

# Import table and keep relevant columns

In [2]:
infile = open(path+'sitzungssummary.pkl','rb')
sitzungen = pickle.load(infile)
infile.close()
sitzungen = sitzungen [['ID','UserID','UserAttribut','Art','HA','beendet','Fehler']]

In [3]:
infile = open(path+'saetze.pkl','rb')
saetze = pickle.load(infile)
infile.close()
saetze = saetze[['satzID','Schwierigkeit']]

In [4]:
infile = open(path+'schueler2020only.pkl','rb')
schueler = pickle.load(infile)
infile.close()
schueler = schueler[['ID','Geschlecht','Klassenstufe','Anmeldeklassenstufe','Aufgaben','Altaufgaben','done']]

In [5]:
infile = open(path+'xmlsaetze.pkl','rb')
xmlsaetze = pickle.load(infile)
infile.close()
xmlsaetze = xmlsaetze[['ID','UserID','UebungsID','Testposition','SatzID','Erstloesung','Schussel','Datum', 'Erfolg','Loesungsnr']]

infile = open(path+'xmlsaetze_archiv.pkl','rb')
xmlsaetze_archiv = pickle.load(infile)
infile.close()
xmlsaetze_archiv = xmlsaetze_archiv[['ID','UserID','UebungsID','Testposition','SatzID','Erstloesung','Schussel','Datum', 'Erfolg','Loesungsnr']]

xmlsaetze = xmlsaetze.append(xmlsaetze_archiv)


# Clean tables

In [6]:
sitzungen = sitzungen.dropna()
sitzungen.rename(columns = {'ID':'UebungsID'}, inplace = True)
# Userattribut: 1=Schüler; 0= Gast/ Lehrer
sitzungen['UserAttribut'] = sitzungen['UserAttribut'].replace(['Schüler'],1)
sitzungen['UserAttribut'] = sitzungen['UserAttribut'].replace(['Gast','Lehrer'],0)
sitzungen[['UserAttribut']] = sitzungen[['UserAttribut']].astype('int16')
#HA: bereinigung
sitzungen['HA'] = sitzungen['HA'].replace(['frHA'],'Self')

In [7]:
schueler.rename(columns = {'ID':'UserID'}, inplace = True)
schueler['done'] = schueler['done'].astype('int16')
#Geschlecht: bereinigung
schueler['Geschlecht'] = schueler['Geschlecht'].replace(['m.'],'m')
schueler['Geschlecht'] = schueler['Geschlecht'].replace(['w.','we'],'w')
schueler['Geschlecht'] = schueler['Geschlecht'].replace(['d','wm'],np.nan)
#Klassenstufe: nutze nur Klassenstufe 5-12
options = ['5', '6','7','8','9','10','11','12'] 
schueler = schueler[schueler['Klassenstufe'].isin(options)] 
schueler['Klassenstufe'] = schueler['Klassenstufe'].astype('int')
#Jahredabei: erstelle aus Klassenstufe - Anmeldeklassenstufe die Spalte Jahredabei
schueler = schueler[schueler['Anmeldeklassenstufe'].isin(options)] 
schueler['Anmeldeklassenstufe'] = schueler['Anmeldeklassenstufe'].astype('int')
schueler['Jahredabei'] = schueler['Klassenstufe'] - schueler['Anmeldeklassenstufe']
#Anzahlaufgaben: erstelle Spalte, die angibt, wie viele Aufgaben dem User aktuell zugeordnet sind
schueler['Aufgaben']=schueler['Aufgaben'].str.split()
schueler['AnzahlAufgaben'] = schueler['Aufgaben'].str.len()
schueler['AnzahlAufgaben'] = schueler['AnzahlAufgaben'].replace([np.nan],0)
schueler['AnzahlAufgaben'] = schueler['AnzahlAufgaben'].astype('int')
# Lösche 
schueler= schueler.drop(columns=['Anmeldeklassenstufe','Aufgaben','Altaufgaben'])

In [8]:
#Merge xmlsaetze with saetze
xmlsaetze.rename(columns = {'SatzID':'satzID'}, inplace = True)
xmlsaetze= pd.merge(xmlsaetze, saetze, on='satzID', how='inner')
xmlsaetze['Datum']= pd.to_datetime(xmlsaetze['Datum'])
xmlsaetze['Uhrzeit'] = pd.DatetimeIndex(xmlsaetze['Datum']).hour
xmlsaetze['Wochentag'] = pd.DatetimeIndex(xmlsaetze['Datum']).dayofweek
xmlsaetze['Kalenderwoche'] = pd.DatetimeIndex(xmlsaetze['Datum']).strftime("%V")
xmlsaetze['Monat'] = pd.DatetimeIndex(xmlsaetze['Datum']).month
xmlsaetze['Tag'] = pd.DatetimeIndex(xmlsaetze['Datum']).day

def f(row):
    if row['Uhrzeit'] > 14:
        val = 0
    elif row['Uhrzeit'] < 8:
        val = 0
    else:
        val = 1
    return val

xmlsaetze['ist_Schulzeit'] = xmlsaetze.apply(f, axis=1)
xmlsaetze = xmlsaetze.drop(columns='UserID')


In [15]:
xmlsaetze = xmlsaetze[(xmlsaetze.Testposition == 'version') | (xmlsaetze.Testposition == 'pruefung') | (xmlsaetze.Testposition == 'training')]
xmlsaetze = xmlsaetze[(xmlsaetze.Monat == 3) | (xmlsaetze.Monat == 4)]#|(xmlsaetze.Monat == 5) | (xmlsaetze.Monat == 6)

In [10]:
# variable mehrfach falsch
xmlsaetze['Loesungsnr']=xmlsaetze['Loesungsnr'].str.split()
xmlsaetze['MehrfachFalsch'] = xmlsaetze['Loesungsnr'].str.len()
xmlsaetze['MehrfachFalsch'] = xmlsaetze['MehrfachFalsch'].replace([np.nan],0)
xmlsaetze['MehrfachFalsch'] = xmlsaetze['MehrfachFalsch'].astype('int')
xmlsaetze['MehrfachFalsch'] = xmlsaetze['MehrfachFalsch'] - 1

xmlsaetze.to_pickle('xmlsaetze_preprocessed.pkl')

In [3]:
infile = open('xmlsaetze_preprocessed.pkl','rb')
xmlsaetze = pickle.load(infile)
infile.close()

xmlsaetze['group'] = xmlsaetze.UebungsID % 10

xmlsaetze1 = xmlsaetze[xmlsaetze.group == 1]
xmlsaetze2 = xmlsaetze[xmlsaetze.group == 2]
xmlsaetze3 = xmlsaetze[xmlsaetze.group == 3]
xmlsaetze4 = xmlsaetze[xmlsaetze.group == 4]
xmlsaetze5 = xmlsaetze[xmlsaetze.group == 5]
xmlsaetze6 = xmlsaetze[xmlsaetze.group == 6]
xmlsaetze7 = xmlsaetze[xmlsaetze.group == 7]
xmlsaetze8 = xmlsaetze[xmlsaetze.group == 8]
xmlsaetze9 = xmlsaetze[xmlsaetze.group == 9]
alle = [xmlsaetze1, xmlsaetze2, xmlsaetze3, xmlsaetze4, xmlsaetze5, xmlsaetze6, xmlsaetze7, xmlsaetze8, xmlsaetze9]

In [ ]:
# Ordernumber and Lücke finden
cnt = 0

for x in alle:

    cnt = cnt +1
    
    num = str(cnt)
    print(num)
    sitzungen = x.UebungsID.unique()
    ordernumbers =pd.DataFrame(columns=['OrderNumber', 'ID'])
    for i in sitzungen:
        subset = x.loc[x['UebungsID'] == i]
        subset = subset.sort_values(by='Datum')
        subset = subset.reset_index(drop=True)
        subset.reset_index(inplace=True)
        subset = subset.rename(columns = {'index':'OrderNumber'})
        subset = subset[['OrderNumber','ID']]
        ordernumbers = ordernumbers.append(subset)

    x = pd.merge(x,ordernumbers,on='ID', how='left')
    x['OrderNumber'] = x['OrderNumber'] + 1
    x.to_pickle('xmlsaetze'+num+'_ordernumber.pkl')

    x['Time'] = pd.DatetimeIndex(x['Datum']).strftime("%X")

    # Finde die Sessions, in denen eine Lücke von Monaten / Tagen oder mehr als 45 Minuten Uhrzeit ist

    df_gr = x.groupby('UebungsID').agg({'Monat': ['min', 'max'], 'Tag': ['min', 'max'], 'Time': ['min', 'max']})
    df_gr = df_gr.reset_index()

    def f(row):
        if row.Monat['min'] != row.Monat['max']:
            val = 1
        elif  row.Tag['min'] != row.Tag['max']:
            val = 1
        elif  row.Time['min'] != row.Time['max']:
            if ((pd.to_datetime(row.Time['min']) + pd.to_timedelta(45, unit='m')) < pd.to_datetime(row.Time['max'])):
                val = 1
            else:
                val = 0
        else:
            val = 0
        return val

    df_gr['luecke'] = df_gr.apply(f, axis=1)

    df_gr.to_pickle('xmlsaetze'+num+'_luecke.pkl')

Variable Session Nr hinzurüfen -> für jede der 10 Teile einzeln ausfuehren!

In [38]:
infile = open('files/xmlsaetze1_luecke.pkl','rb')
xmlsaetze1_luecke = pickle.load(infile)
infile.close()
xmlsaetze1_luecke.columns = xmlsaetze1_luecke.columns.droplevel(1)
xmlsaetze1_luecke = xmlsaetze1_luecke[['UebungsID','luecke']]

infile = open('files/xmlsaetze1_ordernumber.pkl','rb')
xmlsaetze1_ordernumber = pickle.load(infile)
infile.close()

xmlsaetze1_merged = pd.merge(xmlsaetze1_ordernumber, xmlsaetze1_luecke, on='UebungsID')

xmlsaetze1_luecke0 = xmlsaetze1_merged[xmlsaetze1_merged.luecke==0]
xmlsaetze1_luecke1 = xmlsaetze1_merged[xmlsaetze1_merged.luecke==1]

xmlsaetze1_luecke0['sessionNr'] = 1

#Variable Sessionnr hinzufügen

def f(aktuelltag, aktuellmonat, aktuelluhrzeit, vergleichtag, vergleichmonat, vergleichuhrzeit):
    if aktuellmonat.values[0] != vergleichmonat.values[0]:
        val = 1
    elif  aktuelltag.values[0] != vergleichtag.values[0]:
        val = 1
    elif  aktuelluhrzeit.values[0] != vergleichuhrzeit.values[0]:
        if (vergleichuhrzeit.values[0]+3) < aktuelluhrzeit.values[0]:
            val = 1
        else:
            val= 0
    else:
        val = 0
    return val

errorlist = []

xml_with_breaks = xmlsaetze1_luecke1
xml_with_breaks = xml_with_breaks.astype({'UebungsID': str})
xml_with_breaks['sessionNr'] = 0
sitzungen = xml_with_breaks.UebungsID.unique()
for x in sitzungen:
    try:
        anzahl = len(xml_with_breaks.loc[xml_with_breaks['UebungsID'] == x])

        nr1 = xml_with_breaks.loc[(xml_with_breaks['UebungsID'] == x) & (xml_with_breaks.OrderNumber == 1)] 
        id1 = nr1.ID
        xml_with_breaks.loc[(xml_with_breaks.ID.values == id1.values), 'sessionNr'] = 1

        vergleichmonat = xml_with_breaks['Monat'].loc[(xml_with_breaks['UebungsID'] == x) & (xml_with_breaks.OrderNumber == 1)] 
        vergleichtag = xml_with_breaks['Tag'].loc[(xml_with_breaks['UebungsID'] == x) & (xml_with_breaks.OrderNumber == 1)]    
        vergleichuhrzeit = xml_with_breaks['Uhrzeit'].loc[(xml_with_breaks['UebungsID'] == x) & (xml_with_breaks.OrderNumber == 1)] 

        session = 1

        for y in range(anzahl-1):
            a = y+2
            nr = xml_with_breaks.loc[(xml_with_breaks['UebungsID'] == x) & (xml_with_breaks.OrderNumber == a)]
            this_id = nr.ID

            aktuelltag = nr['Tag']
            aktuellmonat = nr['Monat']
            aktuelluhrzeit = nr['Uhrzeit']

            is_break = f(aktuelltag, aktuellmonat, aktuelluhrzeit, vergleichtag, vergleichmonat, vergleichuhrzeit)

            if is_break == 1:
                xml_with_breaks.loc[(xml_with_breaks.ID.values == this_id.values), 'sessionNr'] = session+1
                session = session+1
            else:
                xml_with_breaks.loc[(xml_with_breaks.ID.values == this_id.values), 'sessionNr'] = session

            vergleichmonat = aktuellmonat
            vergleichtag = aktuelltag
            vergleichuhrzeit = aktuelluhrzeit

    except:
        print(x)

xml_with_breaks.to_pickle('xmlsaetze1_sessionnr1.pkl')
xmlsaetze1_luecke0.to_pickle('xmlsatze1_sessionnr0.pkl')

Fuer jeden der 10 Teil einzeln ausfuehren

In [21]:
infile = open('xmlsaetze9_sessionnr1.pkl','rb')
xmlsaetze1_sessionnr1 = pickle.load(infile)
infile.close()

infile = open('xmlsaetze9_sessionnr0.pkl','rb')
xmlsaetze1_sessionnr0 = pickle.load(infile)
infile.close()

new_session_nr = xmlsaetze1_sessionnr1.append(xmlsaetze1_sessionnr0)

# Add SessionNr to UebungsID -> macht neue UebungsID pro Session
new_session_nr['sessionNr'] = new_session_nr['sessionNr'].astype(str)
new_session_nr["UebungsID"] = new_session_nr["UebungsID"].map(str) + "_" + new_session_nr["sessionNr"]

# Berechne Ordnernummer neu ( Wenn Session getrennt, muss Session 2 wieder mit ordernr 1 anfangen)
min_ordernr = new_session_nr.groupby('UebungsID')['OrderNumber'].min()
new_order = pd.merge(new_session_nr, min_ordernr, on='UebungsID')
new_order['OrderNumber_y'] = new_order['OrderNumber_y'] - 1
new_order['OrderNumber'] = new_order['OrderNumber_x'] - new_order['OrderNumber_y']

new_order = new_order.drop(columns=['OrderNumber_x', 'OrderNumber_y'])

# Neue Spalte: wenn es sich um die zweite/ dritte/ vierte Session in der Aufgabe handelt -> vorher schonmal abgebrochen = 1
def f(row):
    if row['sessionNr'] != '1':
        val = 1
    else:
        val = 0
    return val

new_order['vorher_abgebrochen'] = new_order.apply(f, axis=1)
new_order.to_pickle('vacay_preprocessed9.pkl')
